In [1]:
import math

FIGURES_PATH = 'out/figures/'
DATASETS_PATH = 'out/datasets/'
DICTS_PATH = 'out/dicts/'
CLUSTERS_PATH = 'out/clusters/'

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
from tqdm.notebook import tqdm
from multiprocesspandas import applyparallel
from pandarallel import pandarallel
import psutil
from sys import getsizeof
import networkx as nx
from scipy.cluster.hierarchy import linkage, fcluster


from netgraph import Graph, InteractiveGraph, EditableGraph

import pickle
import gc 


tqdm.pandas()
from helper import *

In [3]:
def default(mean, count, scatter):
    return (mean + abs(scatter)) / (count ** 2)

def get_dists(dists, count_lower=10, dist_func=default):
    return dict([(i[0], dist_func(i[1][0], i[1][1], i[1][2])) 
                 for i in dists.items() 
                 if (i[1][1] >= count_lower
                     or dist_func(i[1][0], i[1][1], i[1][2]) != 0) 
                 and dist_func(i[1][0], i[1][1], i[1][2]) >= 0])
    


In [4]:
class Metric:
    def __init__(self, method='euclidean', max=100):
        self.method = method
        self.max = max

    def run(self, cluster1, cluster2, dists):
        self.cluster1 = cluster1
        self.cluster2 = cluster2
        self.dists = dists

        if self.method == 'euclidean':
            return self.euclidean()
        if self.method == 'min_dist':
            return self.min_dist()
        if self.method == 'max_dist':
            return self.max_dist()
        if self.method == 'average':
            return self.average()
        if self.method == 'ward':
            return self.ward()

    def _get(self, i, j):
        if i == j:
            return 0.0
        if (i, j) in self.dists:
            return self.dists[(i, j)]
        if (j, i) in self.dists:
            return self.dists[(j, i)]
        return self.max


    def euclidean(self):
        n1, n2 = len(self.cluster1), len(self.cluster2)
        s = 0.0
        for i in self.cluster1:
            for j in self.cluster2:
                s += self._get(i, j) ** 2
        return np.sqrt(s)


    def min_dist(self):
        s, mini = 0.0, self.max + 1
        for i in self.cluster1:
            for j in self.cluster2:
                s = self._get(i, j)

                if s < mini:
                    mini = s
        return mini


    def max_dist(self):
        s, maxi = 0.0, -1.0
        for i in self.cluster1:
            for j in self.cluster2:
                s = self._get(i, j)

                if s > maxi:
                    maxi = s
        return maxi


    def average(self):
        n1, n2 = len(self.cluster1), len(self.cluster2)
        s = 0.0
        for i in self.cluster1:
            for j in self.cluster2:
                s += self._get(i, j)
        return s / (n1 * n2)


    def ward(self):
        n1, n2 = len(self.cluster1), len(self.cluster2)
        s_u, s_1, s_2 = 0.0, 0.0, 0.0
        for i in self.cluster1:
            for j in self.cluster2:
                s_u += self._get(i, j) ** 2

        for i in range(n1):
            for j in range(i + 1, n1):
                s_1 += self._get(self.cluster1[i], self.cluster1[j])

        for i in range(n2):
            for j in range(i + 1, n2):
                s_2 += self._get(self.cluster2[i], self.cluster2[j])
        return (s_u - s_1 - s_2) / (n1 + n2)

In [5]:
class ClustersDists:
    def __init__(self, fill=np.inf):
        self.dists = dict()
        self.fill = fill

    def set(self, i, j, val):
        self.dists[i, j] = val

    def get(self, i, j):
        if (i, j) in self.dists:
            return self.dists[i, j]
        if (j, i) in self.dists:
            return self.dists[j, i]
        return self.fill

    def __getitem__(self, key):
        return self.get(key[0], key[1])

    def __setitem__(self, key, value):
        self.set(key[0], key[1], value)

    def min(self):
        return min(self.dists, key=self.dists.get)

    def remove(self, i):
        keys = list(self.dists.keys())
        for key in keys:
            if i in key:
                self.dists.pop(key)

In [6]:
class Clustering:
    def __init__(self, get_dists=get_dists):
        self.get_dists = get_dists
        self.statistics = {
            'min_distances': [],
            'time_of_iter': [],
            'time_of_all': 0.0,
            'count_of_iters': 0.0,
            }

    def get_stats(self):
        self.statistics['time_of_iter'] = np.array(self.statistics['time_of_iter']).mean()
        for k in self.statistics.keys():
            print(f"{k} --- {self.statistics[k]}")
        return self.statistics

    @staticmethod
    def _merge_clusters(cluster1, cluster2):
        merged_cluster = cluster1 + cluster2
        return merged_cluster

    def run(self, dists, k):
        start0 = datetime.now()

        elements = np.unique(list(dists.keys()))[:100_000]
        clusters = [[i] for i in elements]
        iters = len(elements) - k

        clusters_dists = ClustersDists(fill=np.inf)

        print('Starting counting distances between clusters...')

        # manager = multiprocessing.Manager()
        # dists = manager.dict()

        for i in tqdm(range(len(clusters))):
            for j in range(i + 1, len(clusters)):
                clusters_dists.set(i, j, self.metric.run(clusters[i], clusters[j], dists))

        print(getsizeof(clusters_dists))

        print('Starting collapsing closest clusters...')
        for _ in tqdm(range(iters)):
            start = datetime.now()

            i, j = clusters_dists.min()
            # a = np.argmin(clusters_dists)
            # i, j = a // clusters_dists.shape[1], a % clusters_dists.shape[1]

            min_distance = clusters_dists[(i, j)]
            merged_cluster = self._merge_clusters(clusters[i], clusters[j])
            del clusters[j]
            del clusters[i]
            clusters_dists.remove(i)
            clusters_dists.remove(j)

            clusters.append(merged_cluster)

            j = len(clusters) - 1
            for i in range(len(clusters)):
                clusters_dists.set(i, j, self.metric.run(clusters[i], clusters[j], dists))

            self.statistics['min_distances'].append(min_distance)
            self.statistics['time_of_iter'].append(datetime.now() - start)
        self.statistics['count_of_iters'] = iters
        self.statistics['time_of_all'] = datetime.now() - start0

        return clusters

    def run_k_means(self, dists, k, max_iter=10_000):
        # Можно наканпливать minimal_dist, как внутрикластерное расстояние (в агломеративных тоже)
        # Можно сохранять среднее расстояние между кластерами и внутри кластеров, чтобы показывать на графике

        start0 = datetime.now()
        elements = np.unique(sorted(list(dists.keys()))[:25_000])

        clusters = np.random.choice(elements, k, replace=False)
        elements = elements[~np.isin(elements, clusters)]
        clusters = [[c] for c in clusters]


        print('Starting elements splitting by clusters...')
        for e in tqdm(elements):
            minimal_dist = self.metric.max * 5.0
            cluster_index = 0
            for i, c in enumerate(clusters):
                dist = self.metric.run([e], c, dists)
                if dist < minimal_dist:
                    minimal_dist = dist
                    cluster_index = i

            clusters[cluster_index].append(e)


        print('Starting operating over clusters...')
        for _ in tqdm(range(max_iter)):
            self.statistics['count_of_iters'] += 1
            start = datetime.now()
            prev_clusters = clusters.copy()
            for c1 in clusters:
                for pos_el, el in enumerate(c1):
                    minimal_dist = self.metric.max * 5.0
                    cluster_index = 0
                    for i, c in enumerate(clusters):
                        dist = self.metric.run([el], c, dists)
                        if dist < minimal_dist:
                            minimal_dist = dist
                            cluster_index = i

                    self.statistics['min_distances'].append(minimal_dist)

                    del c1[pos_el]
                    clusters[cluster_index].append(el)

            if prev_clusters == clusters:
                print('Clusters stabilizied!')
                self.statistics['time_of_all'] = datetime.now() - start0
                return clusters

            self.statistics['time_of_iter'].append(datetime.now() - start)

        print('Stopped for maximum iterations: {}'.format(max_iter))
        self.statistics['time_of_all'] = datetime.now() - start0
        return clusters


    def fit(self, metric, type='agglomerative', dists_path='date_distances', k=10, max_iter=10_000):
        with open(DATASETS_PATH + dists_path + '.pkl', 'rb') as f:
            self.dists = pickle.load(f)
        dists = self.get_dists(self.dists)
        self.metric = metric
        if type == 'agglomerative':
            return self.run(dists, k)
        else:
            return self.run_k_means(dists, k, max_iter)

In [7]:
c = Clustering()
# clusters_euc = c.fit(metric=Metric('max_dist'), type='k_means', k=10_000, max_iter=10_000)
#
# with open(CLUSTERS_PATH + 'k_means_max_dist.pkl','wb') as f:
#      pickle.dump(clusters_euc, f)

In [ ]:
clusters_euc = c.fit(metric=Metric('min_dist'), type='aglomerative', k=100_000)


with open(CLUSTERS_PATH + 'ward_max_dist.pkl','wb') as f:
     pickle.dump(clusters_euc, f)


clustering...
Starting counting distances between clusters...


  0%|          | 0/26730 [00:00<?, ?it/s]

In [ ]:
c.get_stats()

In [ ]:
# with open(CLUSTERS_PATH + 'k_means_euclidean.pkl','wb') as f:
#      pickle.dump(clusters_euc, f)